1) DCGAN - deep learning taste!

In [ ]:
# Import Libraries
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
import subprocess

In [ ]:
# Install pytorch-fid and dependencies
subprocess.run(["pip", "install", "pytorch-fid", "torch", "torchvision"], check=True)

# Set random seed for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

Q: finish a smal task in step 1

In [ ]:
# Step 1: Load and Preprocess MNIST Dataset
def load_mnist_data():

# 1) Load MNIST dataset (28x28 grayscale images)
# 2) Normalize images to [-1, 1] and reshape for DCGAN
# 3) Create a TensorFlow dataset

In [ ]:
# Step 2: Define the Generator Model
def make_generator_model():
    model = tf.keras.Sequential([
        # Input: Random noise vector (100-dimensional)
        layers.Dense(7*7*256, use_bias=False, input_shape=(100,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Reshape to feature map
        layers.Reshape((7, 7, 256)),

        # Upsample to 14x14
        layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Upsample to 28x28
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),

        # Output: 28x28x1 image
        layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

In [ ]:
# Step 3: Define the Discriminator Model
def make_discriminator_model():
    model = tf.keras.Sequential([
        # Input: 28x28x1 image
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=[28, 28, 1]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        # Downsample
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),

        # Flatten and output probability
        layers.Flatten(),
        layers.Dense(1, activation='sigmoid')
    ])
    return model

In [ ]:
# Step 4: Define Loss Functions
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:

# Step 5: Setup Models and Optimizers
generator = make_generator_model()
discriminator = make_discriminator_model()

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

Question: here what optimizers are we using?

In [ ]:
# Step 6: Training Loop
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
# Step 7: Generate and save images
def generate_and_save_images(model, epoch, test_input, save_dir='generated_images'):
    # Ensure save directory exists
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Generate images
    predictions = model(test_input, training=False)

    # Display 16 images
    fig = plt.figure(figsize=(4, 4))
    for i in range(16):
        plt.subplot(4, 4, i + 1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')

    # Save images
    plt.savefig(f'{save_dir}/image_at_epoch_{epoch:04d}.png')
    plt.show()

In [ ]:

# Step 8: Training function
def train(dataset, epochs):
    for epoch in range(epochs):
        start = time.time()

        # Train on each batch
        for image_batch in dataset:
            gen_loss, disc_loss = train_step(image_batch)

        # Generate and save images after each epoch
        generate_and_save_images(generator, epoch + 1, seed)

        # Print training information
        print(f'Epoch {epoch + 1}, Time: {time.time() - start:.2f} sec, '
              f'Gen Loss: {gen_loss:.4f}, Disc Loss: {disc_loss:.4f}')

    # Generate final images
    generate_and_save_images(generator, epochs, seed)

In [ ]:

# Step 9: Run training
# Parameters
noise_dim = 100
batch_size = 256
epochs = 10
num_examples_to_generate = 16

# Seed for consistent visualization
seed = tf.random.normal([num_examples_to_generate, noise_dim])

# Load data
train_dataset = load_mnist_data()

# Start training
train(train_dataset, epochs)

# Note: Generated images are saved in 'generated_images/' and displayed after each epoch.

Now, you have finished! can you change the epochs to 50 then keep it running have a look at the time series lab?

now the 50 epochs have been finished, what are the difference here?